# Setting optimal MLP's hiperparameters for California Housing dataset

In [2]:
# Importing all necessary libraries
import torch
import torch.nn as nn
import torch.optim as optim
import optuna
import pandas as pd

from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', 70)

In [3]:
# Read preprocessed california dataset
news_df = pd.read_csv('./prep_california_housing.csv')
news_df

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0
...,...,...,...,...,...,...,...,...,...
20428,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0
20429,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0
20430,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0
20431,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0


In [4]:
# Create a Neural Network class with flexible settings
class FlexibleRegressionNN(nn.Module):
    def __init__(self, input_size: int, hidden_layers_sizes: list, dropout_rate: float) -> None:
        super(FlexibleRegressionNN, self).__init__()
        layers = list()

        # Define first and n-1 hidden layers
        neuron_num = input_size
        for size in hidden_layers_sizes:
            layers.append(nn.Linear(neuron_num, size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            neuron_num = size

        layers.append(nn.Linear(neuron_num, 1))
        self.model = nn.Sequential(*layers)

    # Define forward loop for NN
    def forward(self, x):
        return self.model(x)

In [5]:
# Creating function for training and evaluating the model
def train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)
        
        model.eval()
        with torch.no_grad():
            test_loss = 0.0
            for inputs, targets in test_loader:
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                test_loss += loss.item()
            test_loss /= len(test_loader)
        
        print(f'Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}')
    
    return test_loss

# Creating function for selection of best hyperparameters 
def objective(trial):
    # Defining hyperparameters
    num_hidden_layers = trial.suggest_int('num_hidden_layers', 1, 10)
    hidden_layer_sizes = [
        trial.suggest_int(f'layer_{i}_size', 16, 64)
        for i in range(num_hidden_layers)
    ]
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [32, 64, 128, 256])
    epochs = trial.suggest_int('epochs', 5, 20)
    input_size = 8

    prep_df = pd.read_csv('./prep_california_housing.csv')

    X = prep_df.iloc[:, :-1].values
    y = prep_df.iloc[:, -1].values
    y = y.reshape(-1, 1)

    # Data normalization
    scaler_X = MinMaxScaler()
    X_scaled = scaler_X.fit_transform(X)

    scaler_y = MinMaxScaler()
    y_scaled = scaler_y.fit_transform(y)

    # Splitting data into train and test
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

    # Convertation data to PyTorch tensors
    train_features = torch.tensor(X_train, dtype=torch.float32)
    test_features = torch.tensor(X_test, dtype=torch.float32)
    train_targets = torch.tensor(y_train, dtype=torch.float32)
    test_targets = torch.tensor(y_test, dtype=torch.float32)    

    # Creating DataLoader
    train_data = TensorDataset(train_features, train_targets)
    test_data = TensorDataset(test_features, test_targets)

    train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False)

    # Creating a model and defining of optimizer
    model = FlexibleRegressionNN(input_size, hidden_layer_sizes, dropout_rate)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()

    # Training launch and return the test loss
    return train_and_evaluate_model(model, train_loader, test_loader, criterion, optimizer, epochs)

# Settings of searching by Optuna
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Getting the best hyperparameters
print("Best trial:")
trial = study.best_trial
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

[I 2025-03-19 11:14:29,568] A new study created in memory with name: no-name-133dea9b-1fda-4c18-bd64-28816f774ee0
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_10068/679006939.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_10068/679006939.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/8, Train Loss: 0.0869, Test Loss: 0.0489
Epoch 2/8, Train Loss: 0.0431, Test Loss: 0.0287
Epoch 3/8, Train Loss: 0.0350, Test Loss: 0.0308
Epoch 4/8, Train Loss: 0.0315, Test Loss: 0.0288
Epoch 5/8, Train Loss: 0.0297, Test Loss: 0.0297
Epoch 6/8, Train Loss: 0.0278, Test Loss: 0.0261


[I 2025-03-19 11:14:31,507] Trial 0 finished with value: 0.02268604829441756 and parameters: {'num_hidden_layers': 4, 'layer_0_size': 62, 'layer_1_size': 17, 'layer_2_size': 45, 'layer_3_size': 33, 'dropout_rate': 0.3328181767271602, 'learning_rate': 0.0053230161747420505, 'batch_size': 256, 'epochs': 8}. Best is trial 0 with value: 0.02268604829441756.


Epoch 7/8, Train Loss: 0.0261, Test Loss: 0.0256
Epoch 8/8, Train Loss: 0.0248, Test Loss: 0.0227


/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_10068/679006939.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_10068/679006939.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


...

Epoch 14/14, Train Loss: 0.0159, Test Loss: 0.0148


/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_10068/679006939.py:36: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
/var/folders/nd/fsmv2lz93k3_cjwz_1lx75j80000gn/T/ipykernel_10068/679006939.py:37: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)


Epoch 1/13, Train Loss: 0.0307, Test Loss: 0.0205
Epoch 2/13, Train Loss: 0.0214, Test Loss: 0.0181
Epoch 3/13, Train Loss: 0.0200, Test Loss: 0.0172
Epoch 4/13, Train Loss: 0.0187, Test Loss: 0.0175
Epoch 5/13, Train Loss: 0.0182, Test Loss: 0.0196
Epoch 6/13, Train Loss: 0.0178, Test Loss: 0.0174
Epoch 7/13, Train Loss: 0.0176, Test Loss: 0.0165
Epoch 8/13, Train Loss: 0.0171, Test Loss: 0.0202
Epoch 9/13, Train Loss: 0.0174, Test Loss: 0.0173
Epoch 10/13, Train Loss: 0.0169, Test Loss: 0.0163
Epoch 11/13, Train Loss: 0.0168, Test Loss: 0.0156
Epoch 12/13, Train Loss: 0.0167, Test Loss: 0.0158


[I 2025-03-19 11:21:43,561] Trial 99 finished with value: 0.015518359949055593 and parameters: {'num_hidden_layers': 3, 'layer_0_size': 41, 'layer_1_size': 35, 'layer_2_size': 41, 'dropout_rate': 0.10062080248495561, 'learning_rate': 0.002451418834371966, 'batch_size': 32, 'epochs': 13}. Best is trial 85 with value: 0.014225573140720371.


Epoch 13/13, Train Loss: 0.0167, Test Loss: 0.0155
Best trial:
  Value: 0.014225573140720371
  Params:
    num_hidden_layers: 4
    layer_0_size: 63
    layer_1_size: 45
    layer_2_size: 44
    layer_3_size: 55
    dropout_rate: 0.10133800940868326
    learning_rate: 0.0019306007440253094
    batch_size: 32
    epochs: 15


In [6]:
print(f"  Value: {trial.value}")
print("  Params:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

  Value: 0.014225573140720371
  Params:
    num_hidden_layers: 4
    layer_0_size: 63
    layer_1_size: 45
    layer_2_size: 44
    layer_3_size: 55
    dropout_rate: 0.10133800940868326
    learning_rate: 0.0019306007440253094
    batch_size: 32
    epochs: 15
